# Hackaton Desafio Budokai

## Equipe 11

### notebook_v4: A ideia principal nesta versão é modificar os laços para trabalhar com 2017 sendo o X_test e conseguirmos calcular a erro absoluto médio do modelo.

## Bibliotecas:

In [1]:
#importanto bibliotecas

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model, preprocessing

#import statsmodels.api as sm
#from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import StandardScaler
from datetime import datetime

from sklearn.linear_model import Lasso, LassoCV
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split

#### Lendo csv e removendo algumas colunas redundantes:

In [2]:
Y_train=pd.read_csv('shared/bases_budokai_ufpr/produtividade_soja_modelagem.csv')
df=pd.read_csv('shared/bases_budokai_ufpr/agroclimatology_budokai.csv')

In [3]:
Y_train.head()

,nivel,codigo_ibge,name,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,MU,4100103,Abatiá,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0,3480.0
1,MU,4100459,Altamira do Paraná,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0,3818.0
2,MU,4100608,Alto Paraná,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0,2980.0
3,MU,4100707,Alto Piquiri,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0,3700.0
4,MU,4100509,Altônia,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0,2900.0


In [4]:
Y_train['nivel'].unique()

array(['MU'], dtype=object)

In [5]:
#Y_codigo=Y_train['codigo_ibge']

Y_train=Y_train.drop(columns=['nivel', 'name'])

In [6]:
Y_train.head()

,codigo_ibge,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,4100103,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0,3480.0
1,4100459,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0,3818.0
2,4100608,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0,2980.0
3,4100707,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0,3700.0
4,4100509,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0,2900.0


### De acordo com as respostas esperadas temos:

In [7]:
codigo_saida=[4102000,4104303,4104428,4104808,4104907,4109401,4113205,4113700,4113734,4114005,4114401,4117701,4117909,4119608,4119905,4127007,4127403,4127502,4127700,4128005
]

In [8]:
dataset=pd.DataFrame(columns=df.keys())
#dataset.loc[dataset['codigo_ibge']==codigo_saida[j]]

for k in codigo_saida:
    dataset=pd.concat([df.loc[df['codigo_ibge']==k],dataset])

dataset=dataset.reset_index(drop=True)

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131500 entries, 0 to 131499
Data columns (total 38 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   data                 131500 non-null  object 
 1   codigo_ibge          131500 non-null  object 
 2   latitude             131500 non-null  float64
 3   longitude            131500 non-null  float64
 4   T2M_RANGE            131500 non-null  float64
 5   TS                   131500 non-null  float64
 6   T2MDEW               131500 non-null  float64
 7   T2MWET               131500 non-null  float64
 8   T2M_MAX              131500 non-null  float64
 9   T2M_MIN              131500 non-null  float64
 10  T2M                  131500 non-null  float64
 11  QV2M                 131500 non-null  float64
 12  RH2M                 131500 non-null  float64
 13  PRECTOTCORR          131500 non-null  float64
 14  PS                   131500 non-null  float64
 15  WS2M             

#### Transformando o codigo em inteiro novamente.

In [10]:
dataset['codigo_ibge']=list(map(int,dataset['codigo_ibge']))

#df.loc[df['codigo_ibge']==codigo_saida[0]]

### Vamos começar transformando os dados de data em 'datetime' para melhor agrupar em semanas, meses ou anos.

In [11]:
def t_data_day(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.day

def t_data_month(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.month

def t_data_year(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.year
    #return data.day, data.month, data.year

In [12]:
dataset['day']=list(map(t_data_day, dataset['data']))
dataset['month']=list(map(t_data_month, dataset['data']))
dataset['year']=list(map(t_data_year, dataset['data']))

In [13]:
dataset.head()

,data,codigo_ibge,latitude,longitude,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CLRSKY_SFC_SW_DWN,TOA_SW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,day,month,year
0,20030101,4128005,-24.5393,-52.9865,7.99,26.27,23.23,24.83,30.21,22.21,...,30.59,42.56,133.74,165.12,18.27,0.56,2.95,1,1,2003
1,20030102,4128005,-24.5393,-52.9865,6.47,25.70,23.36,24.59,29.33,22.86,...,29.79,42.55,111.66,161.13,15.38,0.48,2.55,2,1,2003
2,20030103,4128005,-24.5393,-52.9865,6.27,24.75,22.78,23.78,28.03,21.76,...,29.66,42.53,89.88,161.48,12.85,0.41,2.12,3,1,2003
3,20030104,4128005,-24.5393,-52.9865,6.38,24.50,22.21,23.29,27.82,21.44,...,28.35,42.51,87.03,153.10,12.10,0.40,2.05,4,1,2003
4,20030105,4128005,-24.5393,-52.9865,8.43,25.62,22.50,23.95,29.68,21.24,...,29.04,42.49,52.05,156.25,7.56,0.24,1.24,5,1,2003


In [14]:
dataset=dataset.drop(dataset.loc[dataset['year']==2003].index).reset_index(drop=True)

### Podemo implementar um sistema para previsões baseado na média dos valores mensais.

In [15]:
dataset=dataset.drop(columns=['data','latitude','longitude','day'])

#### Para cada codigo postal iremos fazer um subgrupo para cada ano e agrupar os meses pela média.

#### Depois concatenamos e criamos o novo dataset.

In [16]:
codigos=dataset['codigo_ibge'].unique()
anos=dataset['year'].unique()

datanew=pd.DataFrame(columns=dataset.keys())

for i in codigos:
    for j in anos:
        aux = dataset.loc[dataset['codigo_ibge']==i].loc[dataset['year']==j].groupby(by='month').agg('mean').reset_index()
        datanew=pd.concat([datanew,aux])
    
datanew=datanew.reset_index(drop=True)

In [17]:
datanew.head()

,codigo_ibge,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M,QV2M,RH2M,...,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,TOA_SW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,month,year
0,4128005,12.313226,27.513548,19.130968,22.804839,32.921290,20.608387,26.477097,14.547742,67.569032,...,23.842258,29.756452,42.040000,129.152258,157.907419,17.224516,0.526452,-29.573548,1,2004
1,4128005,12.955517,27.584483,18.363448,22.427931,33.331379,20.375172,26.492414,13.849310,64.810690,...,23.348966,28.156207,39.736207,126.227931,149.815517,16.812759,0.507586,2.653793,2,2004
2,4128005,13.622258,26.335806,17.173871,21.365806,32.728065,19.107419,25.557097,12.963548,64.225806,...,21.451613,25.462581,35.433226,113.812581,134.426774,15.015484,0.436452,-30.048387,3,2004
3,4128005,10.766667,24.591000,19.021667,21.618333,30.242667,19.473667,24.218667,14.357000,75.240000,...,16.357333,20.808667,29.943000,88.034000,111.137667,11.629667,0.323000,-31.676667,4,2004
4,4128005,9.320645,17.494516,14.863548,16.181613,22.614839,13.293871,17.502581,11.179355,85.648387,...,11.521935,17.273548,25.104194,60.481613,90.170645,7.898387,0.194516,-31.256452,5,2004


### Precisamos colocar os dados dos meses em uma unica linha para alimentar a entrada do modelo.

#### Criando a lista de indices:

In [18]:
col=list(datanew.keys())
col.remove('year')
col.remove('codigo_ibge')
#removendo os dados que não serão escalados

In [19]:
lista_indices=[]

lista_indices.append('codigo_ibge')

for i in range(12):

    
    for j in col:
        
        aux=j+'_'+str(i+1)
        lista_indices.append(aux)
        
lista_indices.append('month')
lista_indices.append('year')

In [20]:
lista_indices

['codigo_ibge',
 'T2M_RANGE_1',
 'TS_1',
 'T2MDEW_1',
 'T2MWET_1',
 'T2M_MAX_1',
 'T2M_MIN_1',
 'T2M_1',
 'QV2M_1',
 'RH2M_1',
 'PRECTOTCORR_1',
 'PS_1',
 'WS2M_1',
 'WS2M_MAX_1',
 'WS2M_MIN_1',
 'WS2M_RANGE_1',
 'WS10M_1',
 'WS10M_MAX_1',
 'WS10M_MIN_1',
 'WS10M_RANGE_1',
 'WS50M_1',
 'WS50M_MAX_1',
 'WS50M_MIN_1',
 'WS50M_RANGE_1',
 'GWETTOP_1',
 'GWETROOT_1',
 'GWETPROF_1',
 'ALLSKY_SFC_SW_DWN_1',
 'CLRSKY_SFC_SW_DWN_1',
 'TOA_SW_DWN_1',
 'ALLSKY_SFC_PAR_TOT_1',
 'CLRSKY_SFC_PAR_TOT_1',
 'ALLSKY_SFC_UVA_1',
 'ALLSKY_SFC_UVB_1',
 'ALLSKY_SFC_UV_INDEX_1',
 'month_1',
 'T2M_RANGE_2',
 'TS_2',
 'T2MDEW_2',
 'T2MWET_2',
 'T2M_MAX_2',
 'T2M_MIN_2',
 'T2M_2',
 'QV2M_2',
 'RH2M_2',
 'PRECTOTCORR_2',
 'PS_2',
 'WS2M_2',
 'WS2M_MAX_2',
 'WS2M_MIN_2',
 'WS2M_RANGE_2',
 'WS10M_2',
 'WS10M_MAX_2',
 'WS10M_MIN_2',
 'WS10M_RANGE_2',
 'WS50M_2',
 'WS50M_MAX_2',
 'WS50M_MIN_2',
 'WS50M_RANGE_2',
 'GWETTOP_2',
 'GWETROOT_2',
 'GWETPROF_2',
 'ALLSKY_SFC_SW_DWN_2',
 'CLRSKY_SFC_SW_DWN_2',
 'TOA_SW_DWN_

#### Iremos criar um dataset menor para cada mês, mudar o nome dos indices para o indice_mes e depois juntar todos em um dataset com todos os indices ordenados por mês.

In [21]:

df_m=datanew[col].loc[datanew['month']==1].rename(columns=dict(zip(col[0:34],lista_indices[1:35]))).drop(columns='month')

a=35
b=69
for i in range(11):
        
    df_m2=datanew[col].loc[datanew['month']==i+2].rename(columns=dict(zip(col[0:34],lista_indices[a:b]))).drop(columns='month')
    a=b
    b=b+34
    df_m=pd.concat([df_m.reset_index(drop=True),df_m2.reset_index(drop=True)], axis=1)
    

In [22]:
df_m

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS2M_MIN_12,WS2M_RANGE_12,WS10M_12,WS10M_MAX_12,WS10M_MIN_12,WS10M_RANGE_12,WS50M_12,WS50M_MAX_12,WS50M_MIN_12,WS50M_RANGE_12
0,12.313226,27.513548,19.130968,22.804839,32.921290,20.608387,26.477097,14.547742,67.569032,2.531935,...,0.652581,0.662258,23.674839,30.573548,42.462258,128.402581,162.105161,17.306452,0.508065,-29.687742
1,8.789677,26.006774,21.852903,23.737419,30.293226,21.501613,25.623871,17.136774,81.254516,7.784839,...,0.610000,0.614516,23.596129,30.226129,42.446129,128.496452,160.859677,17.292581,0.511613,2.659677
2,11.466774,28.138710,20.220000,23.727419,33.260645,21.793871,27.234516,15.484839,69.205484,4.431613,...,0.680000,0.675161,21.499032,29.592903,42.435806,117.794194,157.387097,15.792258,0.492258,2.596129
3,7.730000,25.683871,22.430323,23.973226,29.698710,21.968710,25.514194,17.737419,84.159032,8.712581,...,0.607097,0.613226,23.464194,30.246774,42.436129,127.693871,160.882258,17.193548,0.508065,2.643548
4,10.218710,26.340323,20.238710,22.941290,31.066452,20.848710,25.641935,15.544194,74.416774,3.609032,...,0.566774,0.573226,25.211290,30.488065,42.445484,136.012581,161.500968,18.207742,0.551935,-61.747742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,8.425484,23.434516,20.681290,22.045806,27.865161,19.438710,23.411290,16.423226,85.927097,9.002258,...,0.738065,0.729677,21.962903,30.644194,42.817742,119.750645,163.210645,16.173871,0.500000,2.630323
336,8.140968,23.840000,20.865484,22.313226,28.296452,20.155161,23.761935,16.457742,85.124194,7.655806,...,0.736129,0.728387,21.611290,30.383548,42.809032,118.041935,162.134516,15.931935,0.481290,-29.819677
337,7.663871,22.996129,20.359355,21.692258,27.200645,19.538387,23.025806,15.981613,86.016452,7.799032,...,0.667419,0.667097,25.597419,31.188065,42.804516,138.342258,165.951290,18.576452,0.563548,2.961290
338,9.545484,25.754839,21.086774,23.382581,30.743226,21.199677,25.675161,16.675484,78.064194,4.329032,...,0.770000,0.757419,22.388387,31.019032,42.801935,121.082258,164.555161,16.335484,0.492581,2.577742


#### Reinserindo ano e codigo do ibge:

In [23]:
df_m['codigo_ibge']=datanew['codigo_ibge'].loc[datanew['month']==1].reset_index(drop=True)
df_m['year']=datanew['year'].loc[datanew['month']==1].reset_index(drop=True)

In [24]:
df_m.head()

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS10M_12,WS10M_MAX_12,WS10M_MIN_12,WS10M_RANGE_12,WS50M_12,WS50M_MAX_12,WS50M_MIN_12,WS50M_RANGE_12,codigo_ibge,year
0,12.313226,27.513548,19.130968,22.804839,32.921290,20.608387,26.477097,14.547742,67.569032,2.531935,...,23.674839,30.573548,42.462258,128.402581,162.105161,17.306452,0.508065,-29.687742,4128005,2004
1,8.789677,26.006774,21.852903,23.737419,30.293226,21.501613,25.623871,17.136774,81.254516,7.784839,...,23.596129,30.226129,42.446129,128.496452,160.859677,17.292581,0.511613,2.659677,4128005,2005
2,11.466774,28.138710,20.220000,23.727419,33.260645,21.793871,27.234516,15.484839,69.205484,4.431613,...,21.499032,29.592903,42.435806,117.794194,157.387097,15.792258,0.492258,2.596129,4128005,2006
3,7.730000,25.683871,22.430323,23.973226,29.698710,21.968710,25.514194,17.737419,84.159032,8.712581,...,23.464194,30.246774,42.436129,127.693871,160.882258,17.193548,0.508065,2.643548,4128005,2007
4,10.218710,26.340323,20.238710,22.941290,31.066452,20.848710,25.641935,15.544194,74.416774,3.609032,...,25.211290,30.488065,42.445484,136.012581,161.500968,18.207742,0.551935,-61.747742,4128005,2008


In [25]:
#X_train=df_m.loc[df_m['year']<2018]
#X_test=df_m.loc[df_m['year']>2017]

X_train=df_m.loc[df_m['year']<2017]
X_test=df_m.loc[df_m['year']==2017]
Y_train_1=Y_train.drop(columns=['2017'])

In [26]:
Y_train_1

,codigo_ibge,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,4100103,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0
1,4100459,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0
2,4100608,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0
3,4100707,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0
4,4100509,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,4128559,2300.0,2628.0,2229.0,3000.0,3470.0,1999.0,3385.0,3498.0,1611.0,3743.0,3216.0,3394.0,3500.0
345,4128609,2375.0,1540.0,2167.0,2248.0,2278.0,1892.0,2625.0,3166.0,1941.0,2714.0,2582.0,3111.0,2690.0
346,4128658,2400.0,1900.0,3100.0,3075.0,3113.0,2500.0,3200.0,3350.0,2236.0,2748.0,3048.0,3049.0,1360.0
347,4128708,2477.0,1823.0,2948.0,2835.0,3052.0,3156.0,2971.0,3526.0,1854.0,3403.0,3309.0,3212.0,2836.0


#### Scalando os dados:

In [27]:
colunas_=list(X_train.keys())

colunas_.remove('year')
colunas_.remove('codigo_ibge')
#removendo os dados que não serão escalados

In [28]:
len(colunas_)

408

In [29]:
X_train[colunas_]

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS2M_MIN_12,WS2M_RANGE_12,WS10M_12,WS10M_MAX_12,WS10M_MIN_12,WS10M_RANGE_12,WS50M_12,WS50M_MAX_12,WS50M_MIN_12,WS50M_RANGE_12
0,12.313226,27.513548,19.130968,22.804839,32.921290,20.608387,26.477097,14.547742,67.569032,2.531935,...,0.652581,0.662258,23.674839,30.573548,42.462258,128.402581,162.105161,17.306452,0.508065,-29.687742
1,8.789677,26.006774,21.852903,23.737419,30.293226,21.501613,25.623871,17.136774,81.254516,7.784839,...,0.610000,0.614516,23.596129,30.226129,42.446129,128.496452,160.859677,17.292581,0.511613,2.659677
2,11.466774,28.138710,20.220000,23.727419,33.260645,21.793871,27.234516,15.484839,69.205484,4.431613,...,0.680000,0.675161,21.499032,29.592903,42.435806,117.794194,157.387097,15.792258,0.492258,2.596129
3,7.730000,25.683871,22.430323,23.973226,29.698710,21.968710,25.514194,17.737419,84.159032,8.712581,...,0.607097,0.613226,23.464194,30.246774,42.436129,127.693871,160.882258,17.193548,0.508065,2.643548
4,10.218710,26.340323,20.238710,22.941290,31.066452,20.848710,25.641935,15.544194,74.416774,3.609032,...,0.566774,0.573226,25.211290,30.488065,42.445484,136.012581,161.500968,18.207742,0.551935,-61.747742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,10.642258,23.721935,18.333871,20.804839,28.909355,18.264839,23.276774,14.149677,77.015484,6.255161,...,0.687097,0.683226,23.467742,30.418710,42.834516,128.759677,163.473548,17.360000,0.525161,2.748065
332,10.849355,23.446129,18.836774,21.040000,28.912258,18.062581,23.244839,14.600645,78.423548,4.669677,...,0.668065,0.665484,24.709355,30.882581,42.822581,134.274194,165.051613,18.018065,0.545806,2.874839
333,9.336129,24.036774,20.015484,22.099677,28.984194,19.649032,24.183226,15.638387,79.412581,6.101613,...,0.683548,0.680000,21.390645,30.376452,42.826452,116.979032,162.129032,15.787742,0.468065,2.419032
334,9.156774,24.720968,20.784194,22.766129,29.548065,20.391613,24.748710,16.382258,80.409355,4.708387,...,0.923871,0.901935,16.696774,29.344516,42.818065,92.785806,157.218387,12.595806,0.382258,1.980968


### Podemos criar um pipeline com as etapas do processo para testes:

In [42]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV

In [54]:
modelos_reg=[
    SVC(kernel='rbf',gamma='scale', C=1),
    SVC(kernel="rbf",C=0.025,probability=True),
    AdaBoostRegressor(),
    GradientBoostingRegressor()#learning_rate= 0.04, max_depth= 4, n_estimators=500, subsample= 0.9),
    #RandomForestRegressor(n_estimators=100, random_state=0)
    ]

#pipe.fit(X_train,Y_train)

In [62]:
for modelo in modelos_reg:
    
    pipe= Pipeline([('scaler', StandardScaler()),
                   ('model',modelo)])


    df_error=pd.DataFrame(columns=['codigo_ibge','2017_prev','2017_true'])

    for i in codigo_saida:

        X_train_menor=X_train.loc[X_train['codigo_ibge']==i].drop(columns=['codigo_ibge'])
        Y_train_menor=Y_train_1[Y_train_1['codigo_ibge']==i].drop(columns='codigo_ibge').T

        pipe.fit(X_train_menor, Y_train_menor.values.ravel())


        X_test_menor=X_test.loc[X_test['codigo_ibge']==i].drop(columns=['codigo_ibge'])


        #pipe.fit_predict()
        predicoes=pipe.predict(X_test_menor)


        data_error={'codigo_ibge':i,'2017_prev':float(predicoes),'2017_true':float(Y_train[Y_train['codigo_ibge']==i]['2017'])}


        data_error=pd.DataFrame([data_error])

        df_error=pd.concat([df_error,data_error])

    df_error=df_error.reset_index(drop=True)

    print(modelo)
    print(mean_absolute_error(list(df_error['2017_prev']), list(df_error['2017_true'])))

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
565.15
SVC(C=0.025, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
565.15
AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=None)
593.7383333333333
GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_le

### The best parameters of gradient boosting

In [61]:
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }
grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)

X_train_menor=X_train.drop(columns=['codigo_ibge'])
Y_train_menor=Y_train_1.drop(columns='codigo_ibge').T

grid_GBR.fit(X_train_menor, Y_train_menor)


ValueError: Found input variables with inconsistent numbers of samples: [260, 13]

In [63]:
Y_train_1.drop(columns='codigo_ibge')

,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0
1,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0
2,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0
3,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0
4,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2300.0,2628.0,2229.0,3000.0,3470.0,1999.0,3385.0,3498.0,1611.0,3743.0,3216.0,3394.0,3500.0
345,2375.0,1540.0,2167.0,2248.0,2278.0,1892.0,2625.0,3166.0,1941.0,2714.0,2582.0,3111.0,2690.0
346,2400.0,1900.0,3100.0,3075.0,3113.0,2500.0,3200.0,3350.0,2236.0,2748.0,3048.0,3049.0,1360.0
347,2477.0,1823.0,2948.0,2835.0,3052.0,3156.0,2971.0,3526.0,1854.0,3403.0,3309.0,3212.0,2836.0


In [64]:
print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [65]:
df_error=df_error.reset_index(drop=True)

In [66]:
df_error

,codigo_ibge,2017_prev,2017_true
0,4102000,3245.227026,3967.0
1,4104303,3300.860056,3840.0
2,4104428,3209.501368,4070.0
3,4104808,3547.007947,3900.0
4,4104907,3523.977032,4000.0
5,4109401,3478.437489,4120.0
6,4113205,3191.393096,3350.0
7,4113700,2928.542771,3250.0
8,4113734,2763.609436,3471.0
9,4114005,3285.584331,3350.0


In [35]:
mean_absolute_error(list(df_error['2017_prev']), list(df_error['2017_true']))

697.0195